<a href="https://colab.research.google.com/github/keee1/kaggle/blob/main/Titanic/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

参考にしたサイト：
https://qiita.com/5sigma_AAA/items/0c23907da9330681147b

LightGBM
https://puntadata.com/lightgbm_sample_explain/

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline

df = pd.read_csv('/content/drive/MyDrive/Kaggle/Titanic/train.csv')
print(df.info())
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
df["Age"].hist(figsize = (4,4))

In [ ]:
df_org = df

from sklearn.impute import SimpleImputer

df["Age"] = SimpleImputer().fit_transform(df.iloc[:,[df.columns.get_loc("Age")]])
df.isnull().sum()
df["Age"].hist(figsize = (4,4))

In [ ]:
df["Embarked"].hist(figsize = (4,4))

In [29]:
df['Embarked'] = df['Embarked'].fillna('S')

In [ ]:
df["Fare"] = SimpleImputer().fit_transform(df.iloc[:,[df.columns.get_loc("Fare")]])
df.isnull().sum()
#df["Fare"].hist(figsize = (4,4))

In [ ]:
df["Cabin"].tolist()

In [32]:
df = df.drop(['Cabin', 'Name', 'PassengerId', 'Ticket'], axis=1)

In [ ]:
print(df.isnull().sum())
print(df.dtypes)

In [34]:
df['Sex'] = df['Sex'].apply(lambda x: 1 if x == 'male' else 0)

In [35]:
df['Embarked'] = df['Embarked'].map( {'S': 0 , 'C':1 , 'Q':2}).astype(int)

In [ ]:
print(df.isnull().sum())
print(df.dtypes)

In [ ]:
plt.figure(figsize = (15,15))
sns.heatmap(df.corr(),annot = True)

In [37]:
from sklearn.model_selection import  train_test_split

X = df.drop('Survived', axis=1)
Y = df.Survived

In [38]:
(train_X , test_X , train_Y , test_Y) = train_test_split(X, Y , test_size = 0.3 , random_state = 0)

In [51]:
import lightgbm as lgb

train_data = lgb.Dataset(train_X, label=train_Y, free_raw_data=False)
train_data.save_binary('/content/drive/MyDrive/Kaggle/Titanic/train.bin')
valid_data = lgb.Dataset(test_X, label=test_Y, free_raw_data=False)
valid_data.save_binary('/content/drive/MyDrive/Kaggle/Titanic/valid.bin')

[LightGBM] [Warning] File /content/drive/MyDrive/Kaggle/Titanic/train.bin exists, cannot save binary to it
[LightGBM] [Warning] File /content/drive/MyDrive/Kaggle/Titanic/valid.bin exists, cannot save binary to it


In [ ]:
param = {   'random_state': 0 ,
            'objective': 'binary',
            'metric': 'binary_error',
}
param['metric'] = ['auc', 'binary_logloss']

bst = lgb.LGBMClassifier(**param)
bst.fit(train_X, train_Y)

In [ ]:
num_round = 50
bst = lgb.train(param, train_data, num_round, valid_sets=[valid_data])
bst.save_model('/content/drive/MyDrive/Kaggle/Titanic/model.txt', num_iteration=bst.best_iteration)

In [ ]:
from sklearn.metrics import (roc_curve , auc ,accuracy_score, precision_score)
pred = bst.predict_proba(test_X)
print(pred[:5])
pred = bst.predict(test_X)
print(pred[:5])
fpr, tpr , thresholds = roc_curve(test_Y,pred,pos_label = 1)
auc(fpr,tpr)

In [ ]:
testDf = pd.read_csv('/content/drive/MyDrive/Kaggle/Titanic/test.csv')
testDf.head()
print(testDf.isnull().sum())
testDf_org = testDf

In [101]:
from sklearn.impute import SimpleImputer

passsengerid = testDf['PassengerId']

testDf["Age"] = SimpleImputer().fit_transform(testDf.iloc[:,[testDf.columns.get_loc("Age")]])
testDf["Fare"] = SimpleImputer().fit_transform(testDf.iloc[:,[testDf.columns.get_loc("Fare")]])
testDf['Embarked'] = testDf['Embarked'].fillna('S')

testDf['Sex'] = testDf['Sex'].apply(lambda x: 1 if x == 'male' else 0)
testDf['Embarked'] = testDf['Embarked'].map( {'S': 0 , 'C':1 , 'Q':2}).astype(int)

testDf= testDf.drop(['Cabin','Name','Ticket','PassengerId'],axis =1)
pred = bst.predict(testDf)

submission = pd.DataFrame({'PassengerId':passsengerid, 'Survived':pred})
submission.to_csv('/content/drive/MyDrive/Kaggle/Titanic/submission.csv' , index = False)